In [1]:
import pandas as pd
import chardet
import os

from IPython.display import display, Markdown
from tqdm import tqdm


In [ ]:
# Set your working folder (same as this notebook)
FOLDER = "."

# Define file groups
TRAIN_FILES = [
    "consol_alteryx_28-10-2025_FY25.csv",
    "consol_fy25fy26_27-10-2025.csv",
    "consol_alteryx_28-10-2025_FY24"
]
TEST_FILE = "consol_alteryx_28-10-2025_FY26.csv"

In [14]:
# ------------------------------------------------
# 🧠 Helper Functions
# ------------------------------------------------

def detect_encoding(file_path, n_bytes=200000):
    """Detect encoding using chardet."""
    with open(file_path, "rb") as f:
        result = chardet.detect(f.read(n_bytes))
    display(Markdown(f"**Detected encoding for `{os.path.basename(file_path)}` → `{result['encoding']}` (confidence: {result['confidence']:.2f})**"))
    return result["encoding"]

def find_separator(file_path, encoding):
    # Your files use '±' (U+00B1) as the delimiter
    sep = "±"
    temp = pd.read_csv(file_path, encoding=encoding, sep=sep, nrows=5, engine="python")
    display(Markdown(f"✅ Using separator **`{sep}` (±)** — detected **{temp.shape[1]} columns**"))
    return sep

def load_large_csv(file_path, encoding, sep, chunksize=100000):
    """Read large CSV safely using chunks."""
    chunks = []
    total_rows = 0
    for chunk in tqdm(pd.read_csv(file_path, encoding=encoding, sep=sep, chunksize=chunksize, engine="python"),
                      desc=f"Loading {os.path.basename(file_path)}"):
        chunks.append(chunk)
        total_rows += len(chunk)
    df = pd.concat(chunks, ignore_index=True)
    display(Markdown(f"✅ Loaded `{os.path.basename(file_path)}` → shape: **{df.shape}**"))
    return df

def show_data_summary(df, name="DataFrame"):
    """Display shape, sample, info, missing %, and numeric stats."""
    display(Markdown(f"### 📊 {name} Overview"))
    display(df.head(3))
    display(Markdown(f"**Shape:** {df.shape[0]:,} rows × {df.shape[1]:,} columns"))
    display(Markdown("#### 🔍 Data Info"))
    display(df.info(verbose=False))
    display(Markdown("#### 🧮 Missing Values (%)"))
    display((df.isnull().mean() * 100).round(2).sort_values(ascending=False).head(10))
    display(Markdown("#### 📈 Numeric Columns Summary"))
    display(df.describe(include='number').T.head(10))


In [13]:
file_path = "consol_alteryx_28-10-2025_FY25.csv"

with open(file_path, "rb") as f:
    raw = f.read(500)
print(raw[:200])

b'Year\xb1Engagement ID\xb1Eng ID\xb1MS\xb1Location\xb1Engagement\xb1Release Date\xb1Last Invoice Issued Date\xb1Last Time Charged Date\xb1Last Expenses Charged Date\xb1Opportunity ID\xb1Opportunity\xb1Opportunity Partner ID\xb1Opportunity P'


In [17]:
train_dfs = []

for file in TRAIN_FILES:
    path = os.path.join(FOLDER, file)
    enc = "mac_roman"
    sep = "±"
    df = load_large_csv(path, enc, sep)
    train_dfs.append(df)

train_df = pd.concat(train_dfs, ignore_index=True)
show_data_summary(train_df, "TRAIN DATA (Merged)")



Loading consol_alteryx_28-10-2025_FY25.csv: 0it [00:00, ?it/s]

Loading consol_alteryx_28-10-2025_FY25.csv: 11it [02:12, 12.01s/it]


✅ Loaded `consol_alteryx_28-10-2025_FY25.csv` → shape: **(1062745, 170)**

Loading consol_fy25fy26_27-10-2025.csv: 14it [02:35, 11.07s/it]


✅ Loaded `consol_fy25fy26_27-10-2025.csv` → shape: **(1363637, 134)**

...

None

#### 🧮 Missing Values (%)

MS Tagging                  100.00
Assurance Plan               99.91
Prorata Plan                 99.91
Plan                         99.91
Assurance PY Actual          99.91
Partner SSL                  99.06
Partner MS                   99.06
Audit Partner's Location     97.14
Audit Partner's MS           97.11
Audit Partner's GPN          90.80
dtype: float64

#### 📈 Numeric Columns Summary

,count,mean,std,min,25%,50%,75%,max
Eng ID,2417319.0,6.101856e+07,1.059471e+07,12345678.0,45511912.0,67653538.0,68291296.0,69439800.0
Opportunity Partner ID,2407173.0,3.732495e+07,1.645611e+07,1022886.0,45000295.0,45001385.0,45004492.0,51001082.0
Engagement Partner GUI,2417251.0,3.843853e+07,1.545715e+07,1007165.0,45000295.0,45001425.0,45005290.0,45817253.0
Engagement Manager GUI,2417251.0,2.033869e+07,2.127735e+07,1070696.0,2184112.0,3106296.0,45802904.0,45817567.0
GFIS CID,2379147.0,1.997944e+07,2.866421e+07,0.0,0.0,0.0,60703416.0,61892939.0
Engagement Service Code,2417237.0,1.014040e+04,1.499018e+02,10001.0,10067.0,10067.0,10119.0,11359.0
Transaction Service Code,2417237.0,1.014040e+04,1.499018e+02,10001.0,10067.0,10067.0,10119.0,11359.0
Charged Hours ETD,2417251.0,1.007250e+03,2.889845e+03,-1394.0,79.0,314.0,926.0,202701.1
Charged Hours FYTD,2417251.0,1.788745e+02,1.088892e+03,-3168.0,0.0,0.0,24.0,103014.0
Charged Hours MTD,2417237.0,1.964324e+01,1.563148e+02,-10043.0,0.0,0.0,0.0,21579.5


In [18]:
train_df.head

<bound method NDFrame.head of          Year Engagement ID      Eng ID   MS   Location  \
0        FY25    E-67195917  67195917.0  NaN        NaN   
1        FY25    E-67407732  67407732.0  NaN        NaN   
2        FY25    E-45286932  45286932.0  NaN        NaN   
3        FY25    E-45146567  45146567.0  NaN        NaN   
4        FY25    E-42655917  42655917.0  NaN        NaN   
...       ...           ...         ...  ...        ...   
2426377  FY26            57         NaN  TMT  Bangalore   
2426378  FY26            58         NaN  TMT    Chennai   
2426379  FY26            59         NaN  TMT      Delhi   
2426380  FY26            60         NaN  TMT  Hyderabad   
2426381  FY26            61         NaN  TMT     Mumbai   

                                   Engagement         Release Date  \
0        AB173-Signify Stat Audit YE Dec 2022  2023-02-23 00:00:00   
1                CB104 AWSBPL Stat Audit FY23  2023-05-03 00:00:00   
2              SC TIC Mfg+Ind Group Report 22  2023

In [19]:
# 🔍 Check head and tail for the merged FY25 training data
print("=== TRAIN DATA (FY25) — HEAD ===")
display(train_df.head(5))

print("\n=== TRAIN DATA (FY25) — TAIL ===")
display(train_df.tail(5))


=== TRAIN DATA (FY25) — HEAD ===


,Year,Engagement ID,Eng ID,MS,Location,Engagement,Release Date,Last Invoice Issued Date,Last Time Charged Date,Last Expenses Charged Date,...,Location Partner,source_file,Year1,Sub_Folder_1,Sub_Folder_2,Month_Serialized,Week,File_Name,Same FY?,Month
0,FY25,E-67195917,67195917.0,NaN,NaN,AB173-Signify Stat Audit YE Dec 2022,2023-02-23 00:00:00,2023-09-15 00:00:00,2024-07-07 00:00:00,2024-07-01 00:00:00,...,NaN,FY25__FY25__01.Jul__Wk 02__Engagement List_Wk 02,FY25,FY25,FY25,01JUL,2,Engagement List_Wk 02,Y,JUL
1,FY25,E-67407732,67407732.0,NaN,NaN,CB104 AWSBPL Stat Audit FY23,2023-05-03 00:00:00,2024-02-21 00:00:00,2024-04-23 00:00:00,2024-07-04 00:00:00,...,NaN,FY25__FY25__01.Jul__Wk 02__Engagement List_Wk 02,FY25,FY25,FY25,01JUL,2,Engagement List_Wk 02,Y,JUL
2,FY25,E-45286932,45286932.0,NaN,NaN,SC TIC Mfg+Ind Group Report 22,2023-01-04 00:00:00,2022-08-23 00:00:00,2022-10-21 00:00:00,2023-12-06 00:00:00,...,NaN,FY25__FY25__01.Jul__Wk 02__Engagement List_Wk 02,FY25,FY25,FY25,01JUL,2,Engagement List_Wk 02,Y,JUL
3,FY25,E-45146567,45146567.0,NaN,NaN,SBL Certification Service 2021,2023-01-04 00:00:00,2022-04-19 00:00:00,2022-12-02 00:00:00,NaN,...,NaN,FY25__FY25__01.Jul__Wk 02__Engagement List_Wk 02,FY25,FY25,FY25,01JUL,2,Engagement List_Wk 02,Y,JUL
4,FY25,E-42655917,42655917.0,NaN,NaN,CB171-Project Marine,2023-01-04 00:00:00,2023-09-19 00:00:00,NaN,2023-04-24 00:00:00,...,NaN,FY25__FY25__01.Jul__Wk 02__Engagement List_Wk 02,FY25,FY25,FY25,01JUL,2,Engagement List_Wk 02,Y,JUL



=== TRAIN DATA (FY25) — TAIL ===


,Year,Engagement ID,Eng ID,MS,Location,Engagement,Release Date,Last Invoice Issued Date,Last Time Charged Date,Last Expenses Charged Date,...,Location Partner,source_file,Year1,Sub_Folder_1,Sub_Folder_2,Month_Serialized,Week,File_Name,Same FY?,Month
2426377,FY26,57,NaN,TMT,Bangalore,NaN,NaN,NaN,NaN,NaN,...,NaN,FY26__FY26__03.Sep__Wk13__Engagement List_Pivo...,FY26,FY26,FY26,03SEP,13,Engagement List_Pivot Wk13 1,Y,SEP
2426378,FY26,58,NaN,TMT,Chennai,NaN,NaN,NaN,NaN,NaN,...,NaN,FY26__FY26__03.Sep__Wk13__Engagement List_Pivo...,FY26,FY26,FY26,03SEP,13,Engagement List_Pivot Wk13 1,Y,SEP
2426379,FY26,59,NaN,TMT,Delhi,NaN,NaN,NaN,NaN,NaN,...,NaN,FY26__FY26__03.Sep__Wk13__Engagement List_Pivo...,FY26,FY26,FY26,03SEP,13,Engagement List_Pivot Wk13 1,Y,SEP
2426380,FY26,60,NaN,TMT,Hyderabad,NaN,NaN,NaN,NaN,NaN,...,NaN,FY26__FY26__03.Sep__Wk13__Engagement List_Pivo...,FY26,FY26,FY26,03SEP,13,Engagement List_Pivot Wk13 1,Y,SEP
2426381,FY26,61,NaN,TMT,Mumbai,NaN,NaN,NaN,NaN,NaN,...,NaN,FY26__FY26__03.Sep__Wk13__Engagement List_Pivo...,FY26,FY26,FY26,03SEP,13,Engagement List_Pivot Wk13 1,Y,SEP


In [ ]:
# Load FY26 Test Data
test_path = os.path.join(FOLDER, TEST_FILE)
enc = "mac_roman"
sep = "±"

test_df = load_large_csv(test_path, enc, sep)
show_data_summary(test_df, "TEST DATA (FY26)")


Exception ignored in: <function tqdm.__del__ at 0x0000024F3D20D120>
Traceback (most recent call last):
  File "c:\Users\FV586VH\Downloads\VSC\Model\.venv\Lib\site-packages\tqdm\std.py", line 1148, in __del__
    self.close()
  File "c:\Users\FV586VH\Downloads\VSC\Model\.venv\Lib\site-packages\tqdm\notebook.py", line 282, in close
    self.disp(bar_style='success', check_delay=False)
AttributeError: 'tqdm_notebook' object has no attribute 'disp'
Loading consol_alteryx_28-10-2025_FY26.csv: 3it [00:28,  9.49s/it]


✅ Loaded `consol_alteryx_28-10-2025_FY26.csv` → shape: **(278942, 188)**

### 📊 TEST DATA (FY26) Overview

,Year,Engagement ID,Eng ID,MS,Location,Engagement,Release Date,Last Invoice Issued Date,Last Time Charged Date,Last Expenses Charged Date,...,System ANSR / Tech Revenue FYTD,source_file,Year1,Sub_Folder_1,Sub_Folder_2,Month_Serialized,Week,File_Name,Same FY?,Month
0,FY26,E-67313430,67313430.0,National,Bangladesh,CB171 - BD Lease Accelerator Support,2023-04-03 00:00:00,2023-07-12 00:00:00,2023-05-19 00:00:00,NaN,...,NaN,FY26__FY26__01.Jul__Wk03__FY25_EngagementList_...,FY26,FY26,FY26,01JUL,3,FY25_EngagementList_Wk03 BI,Y,JUL
1,FY26,E-45403927,45403927.0,National,Bangladesh,SEBL Stat Audit YE 30 Sep 2022,2023-01-04 00:00:00,2023-04-06 00:00:00,2023-06-06 00:00:00,2023-12-06 00:00:00,...,NaN,FY26__FY26__01.Jul__Wk03__FY25_EngagementList_...,FY26,FY26,FY26,01JUL,3,FY25_EngagementList_Wk03 BI,Y,JUL
2,FY26,E-68182933,68182933.0,National,Bangladesh,CB178-Siemens Healthcare Ltd.,2024-04-02 00:00:00,NaN,NaN,NaN,...,NaN,FY26__FY26__01.Jul__Wk03__FY25_EngagementList_...,FY26,FY26,FY26,01JUL,3,FY25_EngagementList_Wk03 BI,Y,JUL


**Shape:** 278,942 rows × 188 columns

...

None

#### 🧮 Missing Values (%)

Partner SSL                 100.00
Assurance PY Actual         100.00
Assurance Plan              100.00
Partner MS                  100.00
MS Tagging                  100.00
Plan                         99.83
Prorata Plan                 99.83
PY closing NUI               98.22
Audit Partner's Location     95.39
Audit Partner's MS           95.35
dtype: float64

#### 📈 Numeric Columns Summary

,count,mean,std,min,25%,50%,75%,max
Eng ID,278357.0,6.245104e+07,1.006641e+07,32310157.0,45546372.0,67897392.0,68635063.0,69439800.0
Opportunity Partner ID,268273.0,3.739433e+07,1.639536e+07,1029616.0,45000295.0,45001386.0,45004758.0,51001082.0
Engagement Partner GUI,278337.0,3.845100e+07,1.546742e+07,1048075.0,45000295.0,45001482.0,45006265.0,45817253.0
Engagement Manager GUI,278337.0,1.938691e+07,2.108393e+07,1070696.0,2184112.0,3062902.0,45802426.0,45817567.0
GFIS CID,278337.0,1.043667e+05,2.524317e+06,0.0,0.0,0.0,0.0,61878441.0
Engagement Service Code,278337.0,1.014555e+04,1.540368e+02,10006.0,10067.0,10067.0,10119.0,11359.0
Transaction Service Code,278337.0,1.014555e+04,1.540368e+02,10006.0,10067.0,10067.0,10119.0,11359.0
Charged Hours ETD,278337.0,1.026791e+03,2.958191e+03,0.0,86.0,330.0,943.8,120462.3
Charged Hours FYTD,278337.0,4.792680e+01,3.388500e+02,-1428.0,0.0,0.0,0.0,31235.5
Charged Hours MTD,278337.0,1.772806e+01,1.470084e+02,-2492.0,0.0,0.0,0.0,12829.0


In [22]:
# 🔍 Check head and tail for FY26 test data
print("=== TEST DATA (FY26) — HEAD ===")
display(test_df.head(5))

print("\n=== TEST DATA (FY26) — TAIL ===")
display(test_df.tail(5))


=== TEST DATA (FY26) — HEAD ===


,Year,Engagement ID,Eng ID,MS,Location,Engagement,Release Date,Last Invoice Issued Date,Last Time Charged Date,Last Expenses Charged Date,...,System ANSR / Tech Revenue FYTD,source_file,Year1,Sub_Folder_1,Sub_Folder_2,Month_Serialized,Week,File_Name,Same FY?,Month
0,FY26,E-67313430,67313430.0,National,Bangladesh,CB171 - BD Lease Accelerator Support,2023-04-03 00:00:00,2023-07-12 00:00:00,2023-05-19 00:00:00,NaN,...,NaN,FY26__FY26__01.Jul__Wk03__FY25_EngagementList_...,FY26,FY26,FY26,01JUL,3,FY25_EngagementList_Wk03 BI,Y,JUL
1,FY26,E-45403927,45403927.0,National,Bangladesh,SEBL Stat Audit YE 30 Sep 2022,2023-01-04 00:00:00,2023-04-06 00:00:00,2023-06-06 00:00:00,2023-12-06 00:00:00,...,NaN,FY26__FY26__01.Jul__Wk03__FY25_EngagementList_...,FY26,FY26,FY26,01JUL,3,FY25_EngagementList_Wk03 BI,Y,JUL
2,FY26,E-68182933,68182933.0,National,Bangladesh,CB178-Siemens Healthcare Ltd.,2024-04-02 00:00:00,NaN,NaN,NaN,...,NaN,FY26__FY26__01.Jul__Wk03__FY25_EngagementList_...,FY26,FY26,FY26,01JUL,3,FY25_EngagementList_Wk03 BI,Y,JUL
3,FY26,E-68506663,68506663.0,National,Bangladesh,CB138-SCCBD Stat Audit 31 Dec 24,2024-08-05 00:00:00,NaN,2025-06-02 00:00:00,2025-01-23 00:00:00,...,NaN,FY26__FY26__01.Jul__Wk03__FY25_EngagementList_...,FY26,FY26,FY26,01JUL,3,FY25_EngagementList_Wk03 BI,Y,JUL
4,FY26,E-45003752,45003752.0,National,Bangladesh,JBL Stat Audit FY 2021,2023-01-04 00:00:00,2023-03-01 00:00:00,2023-06-06 00:00:00,2023-10-17 00:00:00,...,NaN,FY26__FY26__01.Jul__Wk03__FY25_EngagementList_...,FY26,FY26,FY26,01JUL,3,FY25_EngagementList_Wk03 BI,Y,JUL



=== TEST DATA (FY26) — TAIL ===


,Year,Engagement ID,Eng ID,MS,Location,Engagement,Release Date,Last Invoice Issued Date,Last Time Charged Date,Last Expenses Charged Date,...,System ANSR / Tech Revenue FYTD,source_file,Year1,Sub_Folder_1,Sub_Folder_2,Month_Serialized,Week,File_Name,Same FY?,Month
278937,FY26,57,NaN,TMT,Bangalore,NaN,NaN,NaN,NaN,NaN,...,NaN,FY26__FY26__03.Sep__Wk13__Engagement List_Pivo...,FY26,FY26,FY26,03SEP,13,Engagement List_Pivot Wk13 1,Y,SEP
278938,FY26,58,NaN,TMT,Chennai,NaN,NaN,NaN,NaN,NaN,...,NaN,FY26__FY26__03.Sep__Wk13__Engagement List_Pivo...,FY26,FY26,FY26,03SEP,13,Engagement List_Pivot Wk13 1,Y,SEP
278939,FY26,59,NaN,TMT,Delhi,NaN,NaN,NaN,NaN,NaN,...,NaN,FY26__FY26__03.Sep__Wk13__Engagement List_Pivo...,FY26,FY26,FY26,03SEP,13,Engagement List_Pivot Wk13 1,Y,SEP
278940,FY26,60,NaN,TMT,Hyderabad,NaN,NaN,NaN,NaN,NaN,...,NaN,FY26__FY26__03.Sep__Wk13__Engagement List_Pivo...,FY26,FY26,FY26,03SEP,13,Engagement List_Pivot Wk13 1,Y,SEP
278941,FY26,61,NaN,TMT,Mumbai,NaN,NaN,NaN,NaN,NaN,...,NaN,FY26__FY26__03.Sep__Wk13__Engagement List_Pivo...,FY26,FY26,FY26,03SEP,13,Engagement List_Pivot Wk13 1,Y,SEP


In [23]:
# Check how many rows belong to each Fiscal Year
fy_counts = train_df["Year"].value_counts(dropna=False)
display(Markdown("### 📊 Fiscal Year Distribution in TRAIN Data"))
display(fy_counts)


### 📊 Fiscal Year Distribution in TRAIN Data

Year
FY25    2147440
FY26     278942
Name: count, dtype: int64

In [24]:
# ------------------------------------------------
# 🧮 Separate FY25 & FY26 from merged training data
# ------------------------------------------------

# FY25 data (for training)
train_df_fy25 = train_df[train_df["Year"] == "FY25"].copy()

# FY26 data found in training file (leak)
fy26_from_train = train_df[train_df["Year"] == "FY26"].copy()

print(f"FY25 rows (train): {len(train_df_fy25):,}")
print(f"FY26 rows (from training): {len(fy26_from_train):,}")


FY25 rows (train): 2,147,440
FY26 rows (from training): 278,942


In [27]:
fy26_from_train.describe

<bound method NDFrame.describe of          Year Engagement ID      Eng ID        MS    Location  \
2147440  FY26    E-67313430  67313430.0  National  Bangladesh   
2147441  FY26    E-45403927  45403927.0  National  Bangladesh   
2147442  FY26    E-68182933  68182933.0  National  Bangladesh   
2147443  FY26    E-68506663  68506663.0  National  Bangladesh   
2147444  FY26    E-45003752  45003752.0  National  Bangladesh   
...       ...           ...         ...       ...         ...   
2426377  FY26            57         NaN       TMT   Bangalore   
2426378  FY26            58         NaN       TMT     Chennai   
2426379  FY26            59         NaN       TMT       Delhi   
2426380  FY26            60         NaN       TMT   Hyderabad   
2426381  FY26            61         NaN       TMT      Mumbai   

                                   Engagement         Release Date  \
2147440  CB171 - BD Lease Accelerator Support  2023-04-03 00:00:00   
2147441        SEBL Stat Audit YE 30 Sep 2022

In [ ]:
train_df_fy25.describe

<bound method NDFrame.describe of          Year Engagement ID      Eng ID        MS   Location  \
0        FY25    E-67195917  67195917.0       NaN        NaN   
1        FY25    E-67407732  67407732.0       NaN        NaN   
2        FY25    E-45286932  45286932.0       NaN        NaN   
3        FY25    E-45146567  45146567.0       NaN        NaN   
4        FY25    E-42655917  42655917.0       NaN        NaN   
...       ...           ...         ...       ...        ...   
2147435  FY25           NaN         NaN       IIC       Pune   
2147436  FY25           NaN         NaN       IIC  Ahmedabad   
2147437  FY25           NaN         NaN       IIC      Delhi   
2147438  FY25           NaN         NaN       IIC       East   
2147439  FY25           NaN         NaN  National     Mumbai   

                                   Engagement         Release Date  \
0        AB173-Signify Stat Audit YE Dec 2022  2023-02-23 00:00:00   
1                CB104 AWSBPL Stat Audit FY23  2023-05-03

In [ ]:
test_df.shape

(278942, 188)

In [ ]:
train_df_fy25.shape

(2147440, 170)

In [33]:
train_df_fy25.head()

,Year,Engagement ID,Eng ID,MS,Location,Engagement,Release Date,Last Invoice Issued Date,Last Time Charged Date,Last Expenses Charged Date,...,Location Partner,source_file,Year1,Sub_Folder_1,Sub_Folder_2,Month_Serialized,Week,File_Name,Same FY?,Month
0,FY25,E-67195917,67195917.0,NaN,NaN,AB173-Signify Stat Audit YE Dec 2022,2023-02-23 00:00:00,2023-09-15 00:00:00,2024-07-07 00:00:00,2024-07-01 00:00:00,...,NaN,FY25__FY25__01.Jul__Wk 02__Engagement List_Wk 02,FY25,FY25,FY25,01JUL,2,Engagement List_Wk 02,Y,JUL
1,FY25,E-67407732,67407732.0,NaN,NaN,CB104 AWSBPL Stat Audit FY23,2023-05-03 00:00:00,2024-02-21 00:00:00,2024-04-23 00:00:00,2024-07-04 00:00:00,...,NaN,FY25__FY25__01.Jul__Wk 02__Engagement List_Wk 02,FY25,FY25,FY25,01JUL,2,Engagement List_Wk 02,Y,JUL
2,FY25,E-45286932,45286932.0,NaN,NaN,SC TIC Mfg+Ind Group Report 22,2023-01-04 00:00:00,2022-08-23 00:00:00,2022-10-21 00:00:00,2023-12-06 00:00:00,...,NaN,FY25__FY25__01.Jul__Wk 02__Engagement List_Wk 02,FY25,FY25,FY25,01JUL,2,Engagement List_Wk 02,Y,JUL
3,FY25,E-45146567,45146567.0,NaN,NaN,SBL Certification Service 2021,2023-01-04 00:00:00,2022-04-19 00:00:00,2022-12-02 00:00:00,NaN,...,NaN,FY25__FY25__01.Jul__Wk 02__Engagement List_Wk 02,FY25,FY25,FY25,01JUL,2,Engagement List_Wk 02,Y,JUL
4,FY25,E-42655917,42655917.0,NaN,NaN,CB171-Project Marine,2023-01-04 00:00:00,2023-09-19 00:00:00,NaN,2023-04-24 00:00:00,...,NaN,FY25__FY25__01.Jul__Wk 02__Engagement List_Wk 02,FY25,FY25,FY25,01JUL,2,Engagement List_Wk 02,Y,JUL


In [34]:
test_df.head()

,Year,Engagement ID,Eng ID,MS,Location,Engagement,Release Date,Last Invoice Issued Date,Last Time Charged Date,Last Expenses Charged Date,...,System ANSR / Tech Revenue FYTD,source_file,Year1,Sub_Folder_1,Sub_Folder_2,Month_Serialized,Week,File_Name,Same FY?,Month
0,FY26,E-67313430,67313430.0,National,Bangladesh,CB171 - BD Lease Accelerator Support,2023-04-03 00:00:00,2023-07-12 00:00:00,2023-05-19 00:00:00,NaN,...,NaN,FY26__FY26__01.Jul__Wk03__FY25_EngagementList_...,FY26,FY26,FY26,01JUL,3,FY25_EngagementList_Wk03 BI,Y,JUL
1,FY26,E-45403927,45403927.0,National,Bangladesh,SEBL Stat Audit YE 30 Sep 2022,2023-01-04 00:00:00,2023-04-06 00:00:00,2023-06-06 00:00:00,2023-12-06 00:00:00,...,NaN,FY26__FY26__01.Jul__Wk03__FY25_EngagementList_...,FY26,FY26,FY26,01JUL,3,FY25_EngagementList_Wk03 BI,Y,JUL
2,FY26,E-68182933,68182933.0,National,Bangladesh,CB178-Siemens Healthcare Ltd.,2024-04-02 00:00:00,NaN,NaN,NaN,...,NaN,FY26__FY26__01.Jul__Wk03__FY25_EngagementList_...,FY26,FY26,FY26,01JUL,3,FY25_EngagementList_Wk03 BI,Y,JUL
3,FY26,E-68506663,68506663.0,National,Bangladesh,CB138-SCCBD Stat Audit 31 Dec 24,2024-08-05 00:00:00,NaN,2025-06-02 00:00:00,2025-01-23 00:00:00,...,NaN,FY26__FY26__01.Jul__Wk03__FY25_EngagementList_...,FY26,FY26,FY26,01JUL,3,FY25_EngagementList_Wk03 BI,Y,JUL
4,FY26,E-45003752,45003752.0,National,Bangladesh,JBL Stat Audit FY 2021,2023-01-04 00:00:00,2023-03-01 00:00:00,2023-06-06 00:00:00,2023-10-17 00:00:00,...,NaN,FY26__FY26__01.Jul__Wk03__FY25_EngagementList_...,FY26,FY26,FY26,01JUL,3,FY25_EngagementList_Wk03 BI,Y,JUL


In [35]:
# ------------------------------------------------
# 📈 Correlation Analysis — FY25 Training Data
# ------------------------------------------------

# Select only numeric columns
numeric_cols = train_df_fy25.select_dtypes(include=['number']).columns
train_numeric = train_df_fy25[numeric_cols]

# Compute correlation matrix
corr_matrix = train_numeric.corr()

display(Markdown(f"### 🔗 Correlation Matrix — FY25 Training Data ({len(numeric_cols)} numeric columns)"))
display(corr_matrix)


### 🔗 Correlation Matrix — FY25 Training Data (106 numeric columns)

,Eng ID,Opportunity Partner ID,Engagement Partner GUI,Engagement Manager GUI,GFIS CID,Engagement Service Code,Transaction Service Code,Charged Hours ETD,Charged Hours FYTD,Charged Hours MTD,...,FYTD AR Collected,FYTD AR Collected Tax,FYTD Total Collections,CP AR Collected,CP AR Collected Tax,Assurance Plan,Assurance PY Actual,Partner MS,Partner SSL,Week
Eng ID,1.000000,0.041761,0.063314,-0.178513,-0.517365,0.138353,0.138353,-0.081253,0.110982,0.086052,...,0.089815,0.096185,0.094350,0.021140,0.024111,-0.204609,NaN,NaN,NaN,0.081073
Opportunity Partner ID,0.041761,1.000000,0.818383,0.030120,0.016250,-0.008877,-0.008877,-0.010453,-0.000650,-0.004121,...,-0.019103,-0.008030,-0.018658,-0.004914,-0.002535,0.184643,NaN,NaN,NaN,-0.002505
Engagement Partner GUI,0.063314,0.818383,1.000000,0.039060,-0.025918,0.073364,0.073364,-0.027183,-0.012204,-0.012503,...,-0.030295,-0.020875,-0.030517,-0.007740,-0.006202,0.030317,NaN,NaN,NaN,0.000083
Engagement Manager GUI,-0.178513,0.030120,0.039060,1.000000,0.129494,-0.156940,-0.156940,0.055074,-0.003690,-0.005791,...,-0.009100,-0.006275,-0.009165,-0.001917,-0.001288,-0.065730,NaN,NaN,NaN,-0.023634
GFIS CID,-0.517365,0.016250,-0.025918,0.129494,1.000000,-0.261879,-0.261879,0.110698,-0.103534,-0.080394,...,-0.068670,-0.077911,-0.072667,-0.016791,-0.020284,NaN,NaN,NaN,NaN,-0.330197
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Assurance Plan,-0.204609,0.184643,0.030317,-0.065730,NaN,-0.043519,-0.043519,-0.032623,-0.032623,-0.045564,...,NaN,-0.014104,0.030303,-0.049706,-0.060856,1.000000,0.997376,NaN,NaN,-0.010202
Assurance PY Actual,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.997376,1.000000,NaN,NaN,-0.028913
Partner MS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Partner SSL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
test_df.shape